## タスク 2: モデルをトレーニングする

前のタスクでは、組み込みの SageMaker XGBoost アルゴリズムを使用しました。この XGBoost 組み込みアルゴリズムモードには、独自の XGBoost トレーニングスクリプトは組み込まれず、入力データセットで直接実行されます。

このタスクでは、XGBoost をフレームワークとして使用します。XGBoost を SageMaker のフレームワークとして実行すると、独自のトレーニングスクリプトをカスタマイズできるため、K 分割交差検証などの、より高度なシナリオに対してさらに柔軟に対応できるようになります。

トレーニングスクリプトを使用してトレーニングジョブを実行するだけでなく、ハイパーパラメータチューニングジョブを実行してトレーニングを最適化します。ハイパーパラメータは、モデルトレーニング中の学習プロセスに影響を与える高レベルのパラメータです。最適なモデル予測を得るには、ハイパーパラメータ構成を最適化するか、ハイパーパラメータの範囲を設定します。最適な構成を見つけるプロセスは、ハイパーパラメータチューニングと呼ばれます。


### タスク 2.1: 環境を設定する

モデルのトレーニングを開始する前に、必要な依存関係をすべてインストールしてください。


In [ ]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from sagemaker.inputs import TrainingInput
from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# Setting SageMaker variables
sess = sagemaker.Session()
region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sagemaker_role = sagemaker.get_execution_role()
sagemaker_client = boto3.client("sagemaker")

 次に、前と同じように、トレーニングジョブが入力として使用するトレーニングパスと検証パスを設定します。

In [ ]:
# define the bucket and prefix
#read_bucket = "<LAB_BUCKET>"
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        read_bucket = buckets.name
#read_bucket = "labdatabucket-us-west-2-110972467"
read_prefix = "scripts/data" 

# Setting S3 location for read and write operations
train_data_key = f"{read_prefix}/train/adult_data_processed_train_wheaders.csv"
test_data_key = f"{read_prefix}/test/adult_data_processed_test_wheaders.csv"
validation_data_key = f"{read_prefix}/validation/adult_data_processed_validation_wheaders.csv"

write_bucket = read_bucket
write_prefix = "script-mode/data"

model_key = f"{write_prefix}/model"
output_key = f"{write_prefix}/output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
validation_data_uri = f"s3://{read_bucket}/{validation_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"
model_uri = f"s3://{write_bucket}/{model_key}"
output_uri = f"s3://{write_bucket}/{output_key}"
estimator_output_uri = f"s3://{write_bucket}/{write_prefix}/training_jobs"
bias_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/bias"
explainability_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/explainability"

### タスク 2.2: 推定器オブジェクトを設定する

チューニングジョブの設定 

In [ ]:
training_job_name_prefix = "xgbtrain"
tuning_job_name_prefix = "xgbtune" 
xgb_model_name = "xgb-script-mode-model"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

静的ハイパーパラメータ、ハイパーパラメータ範囲、推定器オブジェクトを設定します。

タスク 1 とは異なり、このスクリプトモードのタスクでは、トレーニングジョブ用のカスタムスクリプトを定義します。以下で SageMaker 推定器を定義する際、カスタマイズされた Python スクリプトをエントリポイントとして使用していることに注意してください。少し時間を取って、カスタム xgboost_train.py ファイルを開いてください。

カスタム xgboost_train.py ファイルの中に、相互検証手法を実行しているセクションがあります。これは、標準装備の SageMaker XGBoost トレーニングアルゴリズムでは実現できません。

また、次のセルで定義されているハイパーパラメータの範囲は、最初のタスクの XGBoost 組み込みトレーニングで使用したのと同じ静的ハイパーパラメータであることに注意してください。ただし、今回は、チューナーがこれらのハイパーパラメータのさまざまな値を試して、最適な最終目標メトリクスとの組み合わせを見つけるための範囲を定義します。


In [ ]:
# Set static hyperparameters that will not be tuned
static_hyperparams = {  
                        "eval_metric" : "auc",
                        "objective": "binary:logistic",
                        "num_round": "5"
                      }

# hyperparameter ranges that will be tuned 
hyperparameter_ranges = {
    "max_depth": IntegerParameter(6, 9),
    "eta": ContinuousParameter(0.01, 0.03),
    "gamma": ContinuousParameter(0.5, 0.9),
    "min_child_weight": ContinuousParameter(0.5, 0.9),
    "subsample": ContinuousParameter(0.2, 0.5)
}

# XGBoost Estimator 
xgb_estimator = XGBoost(
                        entry_point="xgboost_train.py",
                        output_path=estimator_output_uri,
                        code_location=estimator_output_uri,
                        hyperparameters=static_hyperparams,
                        role=sagemaker_role,
                        instance_count=train_instance_count,
                        instance_type=train_instance_type,
                        framework_version="1.7-1",
                        base_job_name=training_job_name_prefix
                    )

次に、XGBoost 推定器と定義したハイパーパラメータ範囲を使用するチューナーオブジェクトを作成します。

In [ ]:
objective_metric_name = "validation:auc"

# Setting up tuner object
tuner_config_dict = {
                     "estimator" : xgb_estimator,
                     "max_jobs" : 6,
                     "max_parallel_jobs" : 3,    
                     "objective_metric_name" : objective_metric_name,
                     "hyperparameter_ranges" : hyperparameter_ranges,
                     "base_tuning_job_name" : tuning_job_name_prefix,
                     "strategy" : "Random"
                    }
tuner = HyperparameterTuner(**tuner_config_dict)

チューニングジョブの入力チャンネルを設定してチューナージョブを実行する 

In [ ]:
s3_input_train = TrainingInput(s3_data="s3://{}/{}".format(read_bucket, train_data_key), content_type="csv", s3_data_type="S3Prefix")
s3_input_validation = (TrainingInput(s3_data="s3://{}/{}".format(read_bucket, validation_data_key), content_type="csv", s3_data_type="S3Prefix"))

tuner.fit(inputs={"train": s3_input_train, "validation": s3_input_validation}, include_cls_metadata=False)
tuner.wait()

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i>**正常な出力:** 推定器とハイパーパラメータの設定が適切で、ハイパーパラメータチューニングジョブが正しく完了すると、出力は次のようになるはずです。

```plain
************************
**** EXAMPLE OUTPUT ****
************************

No finished training job found associated with this estimator.Please make sure this estimator is only used for building workflow config (この推定器に関連する終了した Training ジョブが見つかりませんでした。この推定器がワークフロー設定の構築にのみ使用されることを確認してください)
...................................!
!
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **注意:** このトレーニングの所要時間は約 3～4 分です。


パフォーマンスの降順でチューニング結果の要約を実行すると、最も高い Objective Value が一番上に表示されます。 

In [ ]:
df_tuner = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name).dataframe()
df_tuner = df_tuner[df_tuner["FinalObjectiveValue"]>-float('inf')].sort_values("FinalObjectiveValue", ascending=False)
df_tuner

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じる。
- ラボセッションに戻り、ラボを終了する。